In [3]:
print(1+2)

3


In [1]:
gpx_filename = '28may2021.gpx'

In [2]:
import os

from os.path import isfile, join, isdir, dirname, abspath
from os import listdir
import gpxpy
import gpxpy.gpx


def parseActivityDataInGpxData(gpx =  None):
    if not gpx:
        return None

    
    activity = {}
    activity['time'] = gpx.time
    activity['link'] = gpx.link
    tracks = []
    for track_item in gpx.tracks:
        track = {}
        
        track['name'] = track_item.name
        track['type'] = track_item.type
        segments = []
        for segment_item in track_item.segments:
            segment = {}
            points = []
            for point_item in segment_item.points:
                point = []
                #print(point_item)
                #point.append(point.time)
                point.append(float(point_item.latitude))
                point.append(float(point_item.longitude))
                point.append(float(point_item.elevation))
                points.append(point)
            segment['points'] = points
            segments.append(segment)
        track['segments'] = segments
        tracks.append(track)
    activity['tracks'] = tracks
    
    return activity
    

def getActivityDataFromGpxFile(file_path):
    if not isfile(file_path):
        return None
    
    #print('file_path', file_path)
    file_handle = open(file_path, "r")
    gpx_data = gpxpy.parse(file_handle)
    file_handle.close()
    
    activity_data = parseActivityDataInGpxData(gpx_data)
    
    return activity_data


    




In [3]:
data_activity = getActivityDataFromGpxFile(gpx_filename)
points_full = data_activity['tracks'][0]['segments'][0]['points']
 
points = [point[:2] for point in points_full]

In [4]:
print(points)

[[55.585876, 36.566525], [55.585857, 36.566493], [55.585862, 36.566484], [55.585867, 36.566476], [55.585872, 36.566468], [55.585878, 36.566459], [55.585883, 36.566451], [55.585888, 36.566443], [55.585893, 36.566434], [55.585898, 36.566426], [55.585918, 36.566462], [55.585923, 36.566477], [55.585917, 36.566462], [55.585916, 36.566499], [55.585889, 36.566497], [55.585874, 36.566501], [55.585864, 36.566514], [55.585844, 36.566551], [55.585839, 36.566564], [55.585823, 36.566614], [55.585816, 36.566631], [55.585799, 36.566669], [55.585773, 36.566723], [55.585764, 36.566741], [55.585741, 36.566787], [55.585735, 36.566805], [55.585723, 36.566858], [55.585717, 36.566875], [55.585706, 36.566928], [55.585703, 36.566945], [55.585683, 36.567003], [55.585677, 36.567021], [55.585663, 36.567069], [55.585649, 36.567112], [55.585634, 36.567154], [55.585619, 36.5672], [55.585596, 36.567277], [55.585583, 36.567322], [55.585567, 36.567392], [55.585547, 36.567459], [55.585529, 36.567519], [55.585513, 36.56

In [23]:
import statistics


median_0 = statistics.median([point[0] for point in points])
median_1 = statistics.median([point[1] for point in points])
print(median_0, median_1)

55.701058 36.712502


In [26]:
import geopy.distancefrom ipyleaflet import Map, Polyline

line = Polyline(
    locations=points,
    color="red" ,
    fill=False
)

m = Map(center = (median_0, median_1), zoom =12)
m.add(line)
m

Map(center=[55.701058, 36.712502], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [71]:
import geopy.distance


def getDistance(point_s):
    total = 0

    for idx, point in enumerate(point_s):
        if idx == 0:
            continue   
        coords_1 = (point_s[idx-1][0], point_s[idx-1][1])
        coords_2 = (point_s[idx][0], point_s[idx][1])
    
        dist = geopy.distance.geodesic(coords_1, coords_2).m
        total += dist
    return total
    

distance = getDistance(points)

print('Points: ', len(points))
print('Total: ', distance, ' km')

Points:  11589
Total:  45548.371663798825  km


In [95]:

def pointsOrder(point_s):
    points_wight = []
    for idx, point in enumerate(points[1:-1]):
        d_1 = getDistance([points[idx-1], points[idx]])
        d_2 = getDistance([points[idx], points[idx+1]])
        d_3 = getDistance([points[idx-1], points[idx+1]])
        delta = d_1 + d_2 - d_3
        if delta < 0.00001:
            delta = 0.
        #print(d_1, d_2, d_3, '\t', delta)
        
        points_wight.append([idx, delta])

    points_sorted = sorted(points_wight, key=lambda x: x[1])
    return points_sorted


p_order = pointsOrder(points)
print('p_order: len', len(p_order))
print(p_order)


p_order: len 11587
[[3, 0.0], [6, 0.0], [23, 0.0], [39, 0.0], [55, 0.0], [77, 0.0], [122, 0.0], [136, 0.0], [184, 0.0], [246, 0.0], [260, 0.0], [264, 0.0], [361, 0.0], [383, 0.0], [444, 0.0], [457, 0.0], [459, 0.0], [467, 0.0], [484, 0.0], [488, 0.0], [556, 0.0], [646, 0.0], [652, 0.0], [653, 0.0], [654, 0.0], [655, 0.0], [658, 0.0], [688, 0.0], [694, 0.0], [697, 0.0], [864, 0.0], [915, 0.0], [925, 0.0], [972, 0.0], [1006, 0.0], [1073, 0.0], [1092, 0.0], [1146, 0.0], [1150, 0.0], [1183, 0.0], [1215, 0.0], [1298, 0.0], [1314, 0.0], [1346, 0.0], [1347, 0.0], [1447, 0.0], [1505, 0.0], [1590, 0.0], [1727, 0.0], [1747, 0.0], [1769, 0.0], [1774, 0.0], [1795, 0.0], [1822, 0.0], [1846, 0.0], [1848, 0.0], [1860, 0.0], [1875, 0.0], [1983, 0.0], [1990, 0.0], [2039, 0.0], [2054, 0.0], [2058, 0.0], [2097, 0.0], [2114, 0.0], [2179, 0.0], [2212, 0.0], [2215, 0.0], [2239, 0.0], [2242, 0.0], [2301, 0.0], [2312, 0.0], [2388, 0.0], [2398, 0.0], [2508, 0.0], [2516, 0.0], [2522, 0.0], [2609, 0.0], [2623, 0

[[0, 2.923438693542338], [1, 2.923438693542338], [2, 2.923438693542338], [3, 2.923438693542338], [4, 2.923438693542338], [5, 2.923438693542338], [6, 2.923438693542338], [7, 2.923438693542338], [8, 2.923438693542338], [9, 2.923438693542338], [10, 2.923438693542338], [11, 2.923438693542338], [12, 2.923438693542338], [13, 2.923438693542338], [14, 2.923438693542338], [15, 2.923438693542338], [16, 2.923438693542338], [17, 2.923438693542338], [18, 2.923438693542338], [19, 2.923438693542338], [20, 2.923438693542338], [21, 2.923438693542338], [22, 2.923438693542338], [23, 2.923438693542338], [24, 2.923438693542338], [25, 2.923438693542338], [26, 2.923438693542338], [27, 2.923438693542338], [28, 2.923438693542338], [29, 2.923438693542338], [30, 2.923438693542338], [31, 2.923438693542338], [32, 2.923438693542338], [33, 2.923438693542338], [34, 2.923438693542338], [35, 2.923438693542338], [36, 2.923438693542338], [37, 2.923438693542338], [38, 2.923438693542338], [39, 2.923438693542338], [40, 2.92

In [96]:
def compressPoints(percantage=0.1):
    print('🏞 compressPoints: ')

    distance = getDistance(points)
    print('distance: ', distance, 'm')
    points_exculde = []
    max_exclude_distance = distance*percantage
    print('max_exclude_distance: ', max_exclude_distance, 'm')
    
    sum_distance = 0
    idx = 0
    
    print('')
    round = 0 
    compPonts = []
    running = True
    while(running):
        print('🏓 Round: ', round)
        round += 1
        
        p_order = pointsOrder(points)

        for p_idx, p_delta in p_order:
            summed_d = sum_distance + p_delta
            while(summed_d > max_exclude_distance):
                print('🔫 Break From For')
                running = False
                break
                
        
        

    print('All points: ', len(points))
    print('Exclude points: ', len(points_exculde))

    points_selected = []
    for idx, point in enumerate(points):
        if idx in points_exculde:
            continue
            print('Exclude', idx)
        points_selected.append(points[idx])
    print('Selected points: ', len(points_selected))

    return points_selected
    
    

points_draw = compressPoints(0.99)


SyntaxError: invalid syntax (1700204677.py, line 33)

In [62]:
distance_draw = getDistance(points)
print('distance_draw: ', distance_draw)

distance_draw:  45548.371663798825


In [63]:
line = Polyline(locations=points_draw, color="red", fill=False)
m = Map(center = (median_0, median_1), zoom =12)
m.add(line)
m

Map(center=[55.701058, 36.712502], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…